# Classification

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os
import random
import dill
import pickle
from tabulate import tabulate

import sys

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.metrics import precision_recall_curve
from sklearn import metrics
from hmeasure import h_score
try:
  from catboost import CatBoostClassifier
except:
  !pip install catboost
  from catboost import CatBoostClassifier

import time
from datetime import timedelta

In [2]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:  
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/My Drive/MIMIC-III Text Mining/mimim_iii_readmission"

else:
   path_to_repo = os.path.dirname(os.getcwd())

  
print(path_to_repo)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission


In [3]:
path_to_data = os.path.join(path_to_repo, "data","")
print(path_to_data)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\data\


In [4]:
path_to_processed = os.path.join(path_to_data,"processed","")
os.makedirs(path_to_processed, exist_ok=True) # we create the directory if it does not exist
print(path_to_processed)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\data\processed\


In [5]:
path_to_models = os.path.join(path_to_data,"models","")
os.makedirs(path_to_models, exist_ok=True) # we create the directory if it does not exist
print(path_to_models)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\data\models\


In [6]:
# PARAMETERS

session_seed = 42 # set seed for our session
include_val = False # set to True if we want to also create a validation set
tune_models = False # set to True if we want to perform parameter tuning

lemmatize = True # set to false if we want to do stemming
lemma_tag = str(np.where(lemmatize, "_lemma",""))

seed_tag = f'_{session_seed}'

if tune_models:
  tune_tag = '_tuned'
else:
  tune_tag = ''

random.seed(session_seed)

## Train the Models

In [7]:
def load_datasets(method, include_val = True, target = False):
    """
    Function to load train, test and validation set based on the chosen method
    method: string for the processing method we want to load
    include_diag: if we want to load the dataframes with the diagnosis text, default True
    include_test: if we want to load also the test set, default True
    target: if we are importing our target variables
    """
    global path_to_processed
    if target == True: 
        target = 'y_'
    else: 
        target = ''
    # load it back
    train = pd.read_feather(f'{path_to_processed}{target}train_{method}{seed_tag}{lemma_tag}')
    test = pd.read_feather(f'{path_to_processed}{target}test_{method}{seed_tag}{lemma_tag}')
    if include_val == True:
        val = pd.read_feather(f'{path_to_processed}{target}val_{method}{seed_tag}{lemma_tag}')
    else: val = []
    return train, test, val

In [8]:
y_train, y_test, y_val = load_datasets(method = '', include_val = include_val, target = True)

In [9]:
# initialize a dictionary for the results of all the models
final_train = {}
final_val = {}
final_test = {}

In [10]:
model_dict = {
    'log_reg': LogisticRegression(solver = "saga", penalty = 'l1', random_state = session_seed, n_jobs = -1) # default penalty is l2, we do lasso
    , 'dec_tree': DecisionTreeClassifier(random_state = session_seed)
    #, 'bag_tree': BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_estimators = 10, random_state = session_seed, n_jobs = -1)
    , 'rand_for': RandomForestClassifier(random_state = session_seed, n_jobs = -1)
    , 'gboost': GradientBoostingClassifier(random_state = session_seed)
    , 'lightgbm': lgb.LGBMClassifier(random_state = 42, n_jobs = -1, deterministic = True)
    , 'catboost': CatBoostClassifier(random_seed = 42)
}

In [11]:
# PARAMETERS FOR LOGISTIC REGRESSION -------
param_en = {'C': np.logspace(-3, 4, 10), 'l1_ratio':np.linspace(0,1,11) }

# PARAMETERS FOR DECISION TREE -------------
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

param_dec = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

# PARAMETERS FOR RANDOM FOREST -------
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]

# Maximum number of samples per tree
max_sampl = list(np.arange(0.01,1,0.2))
max_sampl.append(None)
# Create the random grid
param_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_samples': max_sampl}

# PARAMETERS FOR GRADIENT BOOSTING --------

learn_rate = list(np.linspace(0, 1, num = 10))

param_gb = {'n_estimators': n_estimators,
            'learning_rate': learn_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsamples': list(np.linspace(0, 1, num = 10))}

# PARAMETERS FOR LIGHTGBM -----------
param_lgb = {'max_depth': max_depth,
             'min_data_in_leaf': min_samples_leaf,
             'num_iterations': n_estimators,
             'learning_rate': learn_rate,
             'colsample_bytree': list(np.linspace(0, 1, num = 10)),
             'subsample': list(np.linspace(0, 1, num = 10)),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

# PARAMETERS FOR CATBOOSTING ------

param_cat = {'n_estimators': n_estimators,
            'learning_rate': learn_rate,
               'rsm': list(np.linspace(0, 1, num = 10)),
               'depth': [int(x) for x in np.linspace(10, 110, num = 11)]
            , 'l2_leaf_reg': [1, 2, 3, 4, 5, 7, 9, 15, 20]}

In [12]:
param_dictionary = {
    'log_reg': param_en
    , 'dec_tree': param_dec
    #, 'bag_tree': BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_estimators = 10, random_state = session_seed, n_jobs = -1)
    , 'rand_for': param_rf
    , 'gboost': param_gb
    , 'lightgbm': param_lgb
    , 'catboost': param_cat
}

In [13]:
method_list = ['frequency', 'onehot','tf_idf', 'svd', 'lda']

In [14]:
def get_main_scores(estimator, df, y):
    """
    Function to get the main scores
    """
    results = {}
    probas = estimator.predict_proba(df)[:,1]
    results['ROC'] = roc_auc_score(y, probas)
    precision, recall, _ = metrics.precision_recall_curve(y, probas)
    results['PrRc'] = metrics.auc(recall, precision)
    results['HScore'] = h_score(y.to_numpy(), estimator.predict(df))
    return results

In [15]:
for model_name, model in model_dict.items(): 
    print(model_name)
    train_res = {}
    val_res = {}
    test_res = {}
    for method in method_list:
        print(method)
        train, test, val = load_datasets(method, include_val = include_val) # we load the dataset we want to use
        start_time = time.monotonic()
        try:
            with open(f'{path_to_models}_{model_name}_{method}{tune_tag}{seed_tag}{lemma_tag}', 'rb') as file:
                estimator = dill.load(file)
            print('Model already trained')
        except:
            if tune_models:
              # if we want to perform parameter tuning we use randomsearchCV
               gridsearch = RandomizedSearchCV(model, param_dictionary[model_name], cv=5, n_jobs=-1, 
                          scoring=['accuracy','roc_auc'], refit = 'roc_auc')
               gridsearch.fit(train, y_train) # we fit our model
               estimator = gridsearch.best_estimator_
            else:
               model.fit(train, y_train) # we fit our model
               estimator = model
            print('Model successfully trained')
            with open(f'{path_to_models}_{model_name}_{method}{tune_tag}{seed_tag}{lemma_tag}', 'wb') as file: # and save the fitted model
                dill.dump(estimator, file)
            print('Model saved')
        end_time = time.monotonic()
        print(timedelta(seconds=end_time - start_time))
        results_train = get_main_scores(estimator, train, y_train.target)
        results_test = get_main_scores(estimator, test, y_test.target)
        train_res[method] = results_train
        test_res[method] = results_test
        print('ROC Training Set: {}'.format(results_train['ROC']))
        print('ROC Test Set: {}'.format(results_test['ROC']))
        if include_val == True:
            results_val = get_main_scores(estimator, val, y_val.target)
            val_res[method] = results_val
            print('ROC Validation Set: {}'.format(results_val['ROC']))
    # finally we add the result lists to our dictionary
    final_train[model_name] = train_res
    final_val[model_name] = val_res
    final_test[model_name] = test_res

log_reg
frequency
Model already trained
0:00:00
ROC Training Set: 0.8404303689537902
ROC Test Set: 0.6081441642286648
onehot
Model already trained
0:00:00
ROC Training Set: 0.9962880824704573
ROC Test Set: 0.6146731339654529
tf_idf
Model already trained
0:00:00
ROC Training Set: 0.7494257691707624
ROC Test Set: 0.7080520269057592
svd
Model already trained
0:00:00
ROC Training Set: 0.6817263851551886
ROC Test Set: 0.6040904911237662
lda
Model already trained
0:00:00
ROC Training Set: 0.7141880272619952
ROC Test Set: 0.6889449515252487
dec_tree
frequency
Model already trained
0:00:00
ROC Training Set: 1.0
ROC Test Set: 0.5539506241761507
onehot
Model already trained
0:00:00
ROC Training Set: 1.0
ROC Test Set: 0.5397233204316474
tf_idf
Model already trained
0:00:00
ROC Training Set: 1.0
ROC Test Set: 0.5418931843756878
svd
Model already trained
0:00:00
ROC Training Set: 1.0
ROC Test Set: 0.5211106123094909
lda
Model already trained
0:00:00
ROC Training Set: 1.0
ROC Test Set: 0.53417547246

103:	learn: 0.1847108	total: 36.9s	remaining: 5m 17s
104:	learn: 0.1846001	total: 37.1s	remaining: 5m 16s
105:	learn: 0.1844582	total: 37.5s	remaining: 5m 16s
106:	learn: 0.1844208	total: 37.8s	remaining: 5m 15s
107:	learn: 0.1843790	total: 38s	remaining: 5m 14s
108:	learn: 0.1843273	total: 38.3s	remaining: 5m 13s
109:	learn: 0.1842878	total: 38.6s	remaining: 5m 12s
110:	learn: 0.1842012	total: 38.8s	remaining: 5m 11s
111:	learn: 0.1841758	total: 39.1s	remaining: 5m 9s
112:	learn: 0.1841051	total: 39.4s	remaining: 5m 8s
113:	learn: 0.1840738	total: 39.6s	remaining: 5m 7s
114:	learn: 0.1840284	total: 39.8s	remaining: 5m 6s
115:	learn: 0.1839094	total: 40.2s	remaining: 5m 6s
116:	learn: 0.1837594	total: 40.6s	remaining: 5m 6s
117:	learn: 0.1836861	total: 40.8s	remaining: 5m 5s
118:	learn: 0.1835048	total: 41.2s	remaining: 5m 4s
119:	learn: 0.1834239	total: 41.5s	remaining: 5m 4s
120:	learn: 0.1833477	total: 41.8s	remaining: 5m 3s
121:	learn: 0.1832417	total: 42.1s	remaining: 5m 2s
122:	l

259:	learn: 0.1758035	total: 1m 19s	remaining: 3m 45s
260:	learn: 0.1757045	total: 1m 19s	remaining: 3m 45s
261:	learn: 0.1756770	total: 1m 19s	remaining: 3m 44s
262:	learn: 0.1756106	total: 1m 20s	remaining: 3m 44s
263:	learn: 0.1755303	total: 1m 20s	remaining: 3m 44s
264:	learn: 0.1755047	total: 1m 20s	remaining: 3m 43s
265:	learn: 0.1754688	total: 1m 20s	remaining: 3m 43s
266:	learn: 0.1754291	total: 1m 21s	remaining: 3m 42s
267:	learn: 0.1753042	total: 1m 21s	remaining: 3m 42s
268:	learn: 0.1752524	total: 1m 21s	remaining: 3m 42s
269:	learn: 0.1750463	total: 1m 22s	remaining: 3m 41s
270:	learn: 0.1749402	total: 1m 22s	remaining: 3m 41s
271:	learn: 0.1748496	total: 1m 22s	remaining: 3m 41s
272:	learn: 0.1748101	total: 1m 22s	remaining: 3m 40s
273:	learn: 0.1746947	total: 1m 23s	remaining: 3m 40s
274:	learn: 0.1746053	total: 1m 23s	remaining: 3m 40s
275:	learn: 0.1744646	total: 1m 23s	remaining: 3m 39s
276:	learn: 0.1743401	total: 1m 24s	remaining: 3m 39s
277:	learn: 0.1743012	total:

412:	learn: 0.1649326	total: 2m 2s	remaining: 2m 53s
413:	learn: 0.1648944	total: 2m 2s	remaining: 2m 53s
414:	learn: 0.1648576	total: 2m 2s	remaining: 2m 52s
415:	learn: 0.1647738	total: 2m 2s	remaining: 2m 52s
416:	learn: 0.1647415	total: 2m 3s	remaining: 2m 52s
417:	learn: 0.1646358	total: 2m 3s	remaining: 2m 51s
418:	learn: 0.1646160	total: 2m 3s	remaining: 2m 51s
419:	learn: 0.1645955	total: 2m 3s	remaining: 2m 51s
420:	learn: 0.1645750	total: 2m 4s	remaining: 2m 50s
421:	learn: 0.1645081	total: 2m 4s	remaining: 2m 50s
422:	learn: 0.1644896	total: 2m 4s	remaining: 2m 50s
423:	learn: 0.1644437	total: 2m 4s	remaining: 2m 49s
424:	learn: 0.1644235	total: 2m 5s	remaining: 2m 49s
425:	learn: 0.1644038	total: 2m 5s	remaining: 2m 49s
426:	learn: 0.1643294	total: 2m 5s	remaining: 2m 48s
427:	learn: 0.1642619	total: 2m 6s	remaining: 2m 48s
428:	learn: 0.1642090	total: 2m 6s	remaining: 2m 48s
429:	learn: 0.1641889	total: 2m 6s	remaining: 2m 47s
430:	learn: 0.1641699	total: 2m 6s	remaining: 

565:	learn: 0.1581279	total: 2m 46s	remaining: 2m 7s
566:	learn: 0.1581108	total: 2m 46s	remaining: 2m 7s
567:	learn: 0.1579997	total: 2m 47s	remaining: 2m 7s
568:	learn: 0.1579353	total: 2m 47s	remaining: 2m 6s
569:	learn: 0.1579193	total: 2m 47s	remaining: 2m 6s
570:	learn: 0.1578535	total: 2m 47s	remaining: 2m 6s
571:	learn: 0.1577746	total: 2m 48s	remaining: 2m 5s
572:	learn: 0.1577519	total: 2m 48s	remaining: 2m 5s
573:	learn: 0.1577369	total: 2m 48s	remaining: 2m 5s
574:	learn: 0.1576657	total: 2m 48s	remaining: 2m 4s
575:	learn: 0.1575525	total: 2m 49s	remaining: 2m 4s
576:	learn: 0.1575368	total: 2m 49s	remaining: 2m 4s
577:	learn: 0.1574314	total: 2m 49s	remaining: 2m 3s
578:	learn: 0.1573336	total: 2m 49s	remaining: 2m 3s
579:	learn: 0.1573175	total: 2m 50s	remaining: 2m 3s
580:	learn: 0.1573017	total: 2m 50s	remaining: 2m 2s
581:	learn: 0.1572844	total: 2m 50s	remaining: 2m 2s
582:	learn: 0.1572680	total: 2m 50s	remaining: 2m 2s
583:	learn: 0.1572526	total: 2m 51s	remaining:

...

301:	learn: 0.1641185	total: 13.5s	remaining: 31.2s
302:	learn: 0.1639941	total: 13.5s	remaining: 31.1s
303:	learn: 0.1638719	total: 13.6s	remaining: 31.1s
304:	learn: 0.1637814	total: 13.6s	remaining: 31s
305:	learn: 0.1636384	total: 13.7s	remaining: 31s
306:	learn: 0.1635521	total: 13.7s	remaining: 30.9s
307:	learn: 0.1634354	total: 13.8s	remaining: 30.9s
308:	learn: 0.1633508	total: 13.8s	remaining: 30.8s
309:	learn: 0.1632240	total: 13.8s	remaining: 30.8s
310:	learn: 0.1631134	total: 13.9s	remaining: 30.7s
311:	learn: 0.1630217	total: 13.9s	remaining: 30.7s
312:	learn: 0.1629061	total: 14s	remaining: 30.6s
313:	learn: 0.1628222	total: 14s	remaining: 30.6s
314:	learn: 0.1627624	total: 14s	remaining: 30.5s
315:	learn: 0.1626681	total: 14.1s	remaining: 30.5s
316:	learn: 0.1625989	total: 14.1s	remaining: 30.5s
317:	learn: 0.1624622	total: 14.2s	remaining: 30.4s
318:	learn: 0.1623977	total: 14.2s	remaining: 30.4s
319:	learn: 0.1622680	total: 14.3s	remaining: 30.4s
320:	learn: 0.1621888	

461:	learn: 0.1495149	total: 20.6s	remaining: 23.9s
462:	learn: 0.1494391	total: 20.6s	remaining: 23.9s
463:	learn: 0.1493148	total: 20.6s	remaining: 23.8s
464:	learn: 0.1492310	total: 20.7s	remaining: 23.8s
465:	learn: 0.1491434	total: 20.7s	remaining: 23.8s
466:	learn: 0.1490728	total: 20.8s	remaining: 23.7s
467:	learn: 0.1489452	total: 20.8s	remaining: 23.7s
468:	learn: 0.1488437	total: 20.9s	remaining: 23.6s
469:	learn: 0.1487831	total: 20.9s	remaining: 23.6s
470:	learn: 0.1486641	total: 20.9s	remaining: 23.5s
471:	learn: 0.1485531	total: 21s	remaining: 23.5s
472:	learn: 0.1484391	total: 21s	remaining: 23.4s
473:	learn: 0.1483503	total: 21.1s	remaining: 23.4s
474:	learn: 0.1482528	total: 21.1s	remaining: 23.3s
475:	learn: 0.1481499	total: 21.1s	remaining: 23.3s
476:	learn: 0.1480835	total: 21.2s	remaining: 23.2s
477:	learn: 0.1480171	total: 21.2s	remaining: 23.2s
478:	learn: 0.1479272	total: 21.3s	remaining: 23.1s
479:	learn: 0.1478309	total: 21.3s	remaining: 23.1s
480:	learn: 0.14

624:	learn: 0.1363308	total: 27.5s	remaining: 16.5s
625:	learn: 0.1362730	total: 27.6s	remaining: 16.5s
626:	learn: 0.1362077	total: 27.6s	remaining: 16.4s
627:	learn: 0.1361623	total: 27.6s	remaining: 16.4s
628:	learn: 0.1361090	total: 27.7s	remaining: 16.3s
629:	learn: 0.1360402	total: 27.7s	remaining: 16.3s
630:	learn: 0.1359804	total: 27.8s	remaining: 16.2s
631:	learn: 0.1359192	total: 27.8s	remaining: 16.2s
632:	learn: 0.1358649	total: 27.9s	remaining: 16.1s
633:	learn: 0.1357966	total: 27.9s	remaining: 16.1s
634:	learn: 0.1357028	total: 27.9s	remaining: 16.1s
635:	learn: 0.1355710	total: 28s	remaining: 16s
636:	learn: 0.1355182	total: 28s	remaining: 16s
637:	learn: 0.1354227	total: 28.1s	remaining: 15.9s
638:	learn: 0.1353420	total: 28.1s	remaining: 15.9s
639:	learn: 0.1352649	total: 28.2s	remaining: 15.8s
640:	learn: 0.1351786	total: 28.2s	remaining: 15.8s
641:	learn: 0.1351088	total: 28.2s	remaining: 15.7s
642:	learn: 0.1350356	total: 28.3s	remaining: 15.7s
643:	learn: 0.134979

784:	learn: 0.1254447	total: 34.3s	remaining: 9.39s
785:	learn: 0.1254037	total: 34.3s	remaining: 9.34s
786:	learn: 0.1253252	total: 34.4s	remaining: 9.3s
787:	learn: 0.1252700	total: 34.4s	remaining: 9.25s
788:	learn: 0.1252193	total: 34.4s	remaining: 9.21s
789:	learn: 0.1251701	total: 34.5s	remaining: 9.17s
790:	learn: 0.1250807	total: 34.5s	remaining: 9.12s
791:	learn: 0.1250313	total: 34.6s	remaining: 9.08s
792:	learn: 0.1249892	total: 34.6s	remaining: 9.03s
793:	learn: 0.1249340	total: 34.6s	remaining: 8.99s
794:	learn: 0.1248998	total: 34.7s	remaining: 8.94s
795:	learn: 0.1248241	total: 34.7s	remaining: 8.9s
796:	learn: 0.1247538	total: 34.8s	remaining: 8.86s
797:	learn: 0.1246862	total: 34.8s	remaining: 8.82s
798:	learn: 0.1246457	total: 34.9s	remaining: 8.77s
799:	learn: 0.1245659	total: 34.9s	remaining: 8.73s
800:	learn: 0.1245414	total: 35s	remaining: 8.68s
801:	learn: 0.1244573	total: 35s	remaining: 8.64s
802:	learn: 0.1243967	total: 35s	remaining: 8.6s
803:	learn: 0.1243389

947:	learn: 0.1155337	total: 41.1s	remaining: 2.26s
948:	learn: 0.1154796	total: 41.2s	remaining: 2.21s
949:	learn: 0.1154128	total: 41.2s	remaining: 2.17s
950:	learn: 0.1153838	total: 41.3s	remaining: 2.13s
951:	learn: 0.1153320	total: 41.3s	remaining: 2.08s
952:	learn: 0.1152903	total: 41.3s	remaining: 2.04s
953:	learn: 0.1152199	total: 41.4s	remaining: 2s
954:	learn: 0.1151828	total: 41.4s	remaining: 1.95s
955:	learn: 0.1151268	total: 41.5s	remaining: 1.91s
956:	learn: 0.1150917	total: 41.5s	remaining: 1.86s
957:	learn: 0.1150363	total: 41.6s	remaining: 1.82s
958:	learn: 0.1150016	total: 41.6s	remaining: 1.78s
959:	learn: 0.1149424	total: 41.7s	remaining: 1.74s
960:	learn: 0.1148864	total: 41.7s	remaining: 1.69s
961:	learn: 0.1148103	total: 41.7s	remaining: 1.65s
962:	learn: 0.1147472	total: 41.8s	remaining: 1.6s
963:	learn: 0.1146925	total: 41.8s	remaining: 1.56s
964:	learn: 0.1146623	total: 41.9s	remaining: 1.52s
965:	learn: 0.1146056	total: 41.9s	remaining: 1.47s
966:	learn: 0.11

In [16]:
try:
    # Then we save all our results
    with open(f'{path_to_models}train_results{tune_tag}{lemma_tag}.pkl', 'rb') as file:
        results_train = pickle.load(file)
    with open(f'{path_to_models}test_results{tune_tag}{lemma_tag}.pkl', 'rb') as file:
        results_test = pickle.load(file)
    if include_val == True:
        with open(f'{path_to_models}val_results{tune_tag}{lemma_tag}.pkl', 'rb') as file:
            results_val = pickle.load(file)
except:
    results_train = {}
    results_val = {}
    results_test = {}

In [17]:
results_train[session_seed] = final_train
results_val[session_seed] = final_val
results_test[session_seed] = final_test

In [18]:
# Then we save all our results
with open(f'{path_to_models}train_results{tune_tag}{lemma_tag}.pkl', 'wb') as file:
    pickle.dump(results_train, file)
with open(f'{path_to_models}test_results{tune_tag}{lemma_tag}.pkl', 'wb') as file:
    pickle.dump(results_test, file)
if include_val == True:
    with open(f'{path_to_models}val_results{tune_tag}{lemma_tag}.pkl', 'wb') as file:
        pickle.dump(results_val, file)

## Display Results

In [20]:
def average_score(dict_res, model, method, score):
    """
    Function to return the average score for a certain combination of model, method and score
    """
    score_avg = 0
    for key in dict_res.keys():
        score_avg += dict_res[key][model][method][score]
    score_avg = score_avg / len(dict_res.keys())
    return score_avg

In [21]:
for score in ['ROC', 'PrRc', 'HScore']:
    final_train = []
    final_test = []
    if include_val: final_val = []
    for model in model_dict.keys():
        train_res = [model]
        test_res = [model]
        if include_val: val_res = [model]
        for method in method_list:
            train_res.append(average_score(results_train, model, method, score))
            test_res.append(average_score(results_test, model, method, score))
            if include_val: val_res.append(average_score(results_val, model, method, score))
        final_train.append(train_res)
        final_test.append(test_res)
        if include_val: final_val.append(val_res)
    print("-"*65)
    print(f'\n{score}\n')
    print('\n Train Results \n')
    print(tabulate(final_train, headers = method_list))
    print("-"*65)
    print('\n Test Results \n')
    print(tabulate(final_test, headers = method_list))
    if include_val: 
        print('\n Val Results \n')
        print(tabulate(final_val, headers = method_list))
        print("-"*65)

-----------------------------------------------------------------

ROC


 Train Results 

            frequency    onehot    tf_idf       svd       lda
--------  -----------  --------  --------  --------  --------
log_reg      0.84043   0.996288  0.749426  0.681726  0.714188
dec_tree     1         1         1         1         1
rand_for     1         1         1         1         1
gboost       0.766028  0.855814  0.829868  0.804993  0.789372
lightgbm     0.995677  0.996257  0.999763  0.999781  0.989041
catboost     0.911767  0.989223  0.979072  0.999668  0.971982
-----------------------------------------------------------------

 Test Results 

            frequency    onehot    tf_idf       svd       lda
--------  -----------  --------  --------  --------  --------
log_reg      0.608144  0.614673  0.708052  0.60409   0.688945
dec_tree     0.553951  0.539723  0.541893  0.521111  0.534175
rand_for     0.700182  0.695747  0.648234  0.658298  0.665418
gboost       0.733978  0.699764  0.